In [ ]:
import pandas as pd
import psycopg2 as psy
import os
from dotenv import load_dotenv

try:

    load_dotenv() # - carregando as variáveis de ambiente para que possam ser utilizadas
    
    conexao = psy.connect(
        dbname = os.getenv("DB_NAME"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor() 

    #execução do comando para trazer os dados da tabela original
    cursor.execute(
        '''SELECT * FROM usuario;'''
    )

    #criando um dataframe apartir dos dados capturados
    df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])
    display(df)#mostrando o dataframe


    cursor.close()
    conexao.close()

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

,id,fk_fabrica,email,senha,nome,tipo_acesso,status,data_criacao,data_nascimento,cargo
0,4f51780f-3770-4d76-8567-802b760646eb,1,gerente.seara@seara.com.br,Seara123,João Mendes,Administrador,Ativo,2025-10-03 11:47:26.806021,1980-05-15,Gerente
1,570e0e1c-d3a8-4fad-a5cb-f512900e540f,1,supervisor.seara@seara.com.br,Seara123,Ana Costa,Supervisor,Ativo,2025-10-03 11:47:26.806021,1985-08-22,Supervisor
2,a66b1520-9cfa-4b63-a5a7-cf8d5b97c9da,2,gerente.sadia@sadia.com.br,Sadia123,Pedro Oliveira,Administrador,Ativo,2025-10-03 11:47:26.806021,1978-12-10,Gerente
3,c36318ea-680d-480d-a78d-2d3a4c03c369,3,gerente.jbs@jbs.com.br,Jbs12345,Fernanda Lima,Administrador,Ativo,2025-10-03 11:47:26.806021,1982-03-30,Gerente
4,a849cb49-c9f2-40da-9727-900cb1067e43,4,supervisor.lins@jbs.com.br,Jbs12345,Ricardo Souza,Supervisor,Ativo,2025-10-03 11:47:26.806021,1990-07-18,Supervisor


In [48]:
#Pseudonimizando as colunas da tabela

df_user_pseudo = df.copy() # - criando uma cópia do dataframe para pseudonimizar de forma eficiente


#Pseudonimização da coluna de id
df_user_pseudo['id'] = [ '*' * (len(str(id))-1)+str(id)[-1] if len(str(id))>1 else "*" for id in df_user_pseudo['id']]
#Pseudonimização da coluna de fk_fabrica
df_user_pseudo['fk_fabrica'] = [ '*' * (len(str(fk))-1)+str(fk)[-1] if len(str(fk))>1 else "*" for fk in df_user_pseudo['fk_fabrica']]
#Pseudonimização da coluna de email
df_user_pseudo['email'] = df_user_pseudo['email'].apply(lambda email: email[:3] + f"{'*' * (email.find('@') -3)}"+email[email.find('@') +1:])
#Pseudonimização da coluna de senha - hashing
#Pseudonimização da coluna de nome
df_user_pseudo['nome'] = df_user_pseudo['nome'].apply(lambda nome: nome[:3]+f'{"*" * (len(nome)-3)}')
#Pseudonimização da coluna de tipo_acesso
df_user_pseudo['tipo_acesso'] = df_user_pseudo['tipo_acesso'].apply(lambda tipo: tipo[:2]+f'{"*" * (len(tipo)-2)}')
#Pseudonimização da coluna de status
df_user_pseudo['status'] = df_user_pseudo['status'].apply(lambda status: status[:1]+f'{"*" * (len(status)-1)}')
#Pseudonimização da coluna de data_criacao
df_user_pseudo['data_criacao'] = df_user_pseudo['data_criacao'].apply(lambda data: str(data)[:10]+f'{"*" * (len(str(data)) -10)}')
#Pseudonimização da coluna de data_nascimento
df_user_pseudo['data_nascimento'] = df_user_pseudo['data_nascimento'].apply(lambda data: f'****/**/'+str(data)[-2:])
#Pseudonimização da coluna de cargo
df_user_pseudo['cargo'] = df_user_pseudo['cargo'].apply(lambda cargo: cargo[:2]+f'{"*" * (len(cargo)-2)}')




In [49]:
#Renomeando as colunas do dataframe
df_user_pseudo.rename(columns={
    "id":"id",
    "fk_fabrica":"fk_fabrica_pseudo",
    "email": "email_pseudo",
    "senha": "senha_hash",
    "nome":"nome_pseudo",
    "tipo_acesso":"tipo_acesso_pseudo",
    "status":"status_pseudo",
    "data_criacao":"data_criacao_pseudo",
    "data_nascimento":"data_nascimento_pseudo",
    "cargo" : "cargo_pseudo"
    
}, inplace=True)

In [ ]:
#Tranferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original
    query = """
    INSERT INTO usuario_pseudo (id, fk_fabrica_pseudo, email_pseudo, senha_hash, nome_pseudo, tipo_acesso_pseudo, status_pseudo, data_criacao_pseudo, data_nascimento_pseudo, cargo_pseudo)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (id) 
    DO UPDATE SET
        fk_fabrica_pseudo = EXCLUDED.fk_fabrica_pseudo,
        email_pseudo = EXCLUDED.email_pseudo,
        nome_pseudo = EXCLUDED.nome_pseudo,
        tipo_acesso_pseudo = EXCLUDED.tipo_acesso_pseudo,
        status_pseudo = EXCLUDED.status_pseudo,
        data_criacao_pseudo = EXCLUDED.data_criacao_pseudo,
        data_nascimento_pseudo = EXCLUDED.data_nascimento_pseudo,
        cargo_pseudo = EXCLUDED.cargo_pseudo
    """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_user_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [51]:
#Tranferindo as informações originais e pseudonimizadas para os arquivos csv
df.to_csv('original_user.csv')
df_user_pseudo.to_csv('pseudo_user.csv')